In [2]:
# The architecture of our CNN is given in Figure 1. The structure
# can be summarized as 28×28×1−26×26×4−100−M,
# where M is the number of classes. The input is a grayscale
# image patch. The size of the image patch is 28×28 pixels. Our
# CNN architecture contains only one convolution layer which
# consists of 4 kernels. The size of each kernel is 3 × 3 pixels.
# Unlike other traditional CNN architecture, the pooling layer is
# not used in our architecture. Then one fully connected layer
# of 100 neurons follows the convolution layer. The last layer
# consists of a logistic regression with softmax which outputs
# the probability of each class, such that

In [66]:
from __future__ import print_function
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision

In [142]:
'''
Get train data
'''
training_data = pickle.load(open('training_data.pkl','r'))

training_data = np.array(training_data)
training_data = training_data[:6000] #Test on first 2000 image segments

X_Train = training_data[:,0]
y_Train = training_data[:,1]

training_data.shape

(6000, 2)

In [100]:
class Net(nn.Module):

    def __init__(self): # DO NOT HARDCODE
        super(Net, self).__init__()
        # 1 input image channel 10x10, 4 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1, 4, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4 * 8 * 8, 100)
        self.fc2 = nn.Linear(100, 2) #Number of classes = 'text'

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [101]:
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear (256 -> 100)
  (fc2): Linear (100 -> 2)
)


In [14]:
# Total number of learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())

6
torch.Size([4, 1, 3, 3])


In [143]:
# Preparing the data
trainloader = DataLoader(training_data.tolist(), batch_size=1, shuffle=True)

In [147]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001)

In [148]:
# Training Phase
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs.unsqueeze(1).float()), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

[1,   200] loss: 0.698
[1,   400] loss: 0.683
[1,   600] loss: 0.713
[1,   800] loss: 0.678
[1,  1000] loss: 0.688
[1,  1200] loss: 0.668
[1,  1400] loss: 0.688
[1,  1600] loss: 0.678
[1,  1800] loss: 0.653
[1,  2000] loss: 0.738
[1,  2200] loss: 0.698
[1,  2400] loss: 0.748
[1,  2600] loss: 0.713
[1,  2800] loss: 0.793
[1,  3000] loss: 0.683
[1,  3200] loss: 0.638
[1,  3400] loss: 0.678
[1,  3600] loss: 0.758
[1,  3800] loss: 0.698
[1,  4000] loss: 0.713
[1,  4200] loss: 0.693
[1,  4400] loss: 0.738
[1,  4600] loss: 0.698
[1,  4800] loss: 0.763
[1,  5000] loss: 0.728
[1,  5200] loss: 0.753
[1,  5400] loss: 0.753
[1,  5600] loss: 0.718
[1,  5800] loss: 0.683
[1,  6000] loss: 0.658
[2,   200] loss: 0.693
[2,   400] loss: 0.718
[2,   600] loss: 0.743
[2,   800] loss: 0.738
[2,  1000] loss: 0.663
[2,  1200] loss: 0.678
[2,  1400] loss: 0.738
[2,  1600] loss: 0.723
[2,  1800] loss: 0.673
[2,  2000] loss: 0.668
[2,  2200] loss: 0.688
[2,  2400] loss: 0.733
[2,  2600] loss: 0.693
[2,  2800] 

In [103]:
# Input to CNN is 10x10

input = Variable(torch.randn(1, 1, 10, 10)) # Here random input is given
out = net(input)
print(out)
# output size is 1x2 because there are 2 labels

Variable containing:
 0.4865  0.5135
[torch.FloatTensor of size 1x2]



In [17]:
# Zero the gradient buffers of all parameters and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1, 2))

In [26]:
# DUMMY CODE
# out = net(input)
target = Variable(torch.arange(0, 2))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

Variable containing:
 0.2909
[torch.FloatTensor of size 1]



In [83]:
training_data = pickle.load(open('training_data.pkl','r'))

training_data = np.array(training_data)

X_Train = training_data[:,0]
y_Train = training_data[:,1]

N = len(X_Train)
H = X_Train[0].shape[0]
W = X_Train[0].shape[1]
trainTensor = torch.LongTensor(N, H, W)
for i in range(N):
    trainTensor[i] = torch.LongTensor(X_Train[i].tolist())
trainTensor = trainTensor.unsqueeze(1)

In [74]:
X_Train[0]

array([[109, 106, 105, 100,  98,  98,  98,  94,  95,  90],
       [ 38,  48,  44,  32,  38,  35,  36,  29,  33,  29],
       [ 24,  22,  24,  24,  21,  22,  22,  20,  19,  21],
       [  8,   8,   7,   8,  10,  10,   8,   8,   9,   8],
       [ 23,  21,  21,   8,  19,   3,  24,  30,  26,  22],
       [  7,   7,   5,   4,   7,   9,   8,   7,   6,   7],
       [  1,   0,   0,   0,   1,   7,   2,   2,   3,   1],
       [ 13,   7,   3,   4,   2,   7,  11,   4,   9,   9],
       [ 18,  23,  36,  18,  28,  28,  37,  47,  39,  50],
       [ 24,  35,  31,  51,  10,  55,  66,  77,  67,  61]], dtype=uint8)

In [84]:
y_Train[0]

0

In [71]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [85]:

dataiter = iter(trainloader)
images, label = dataiter.next()
images


(0 ,.,.) = 
  109  106  105  100   98   98   98   94   95   90
   38   48   44   32   38   35   36   29   33   29
   24   22   24   24   21   22   22   20   19   21
    8    8    7    8   10   10    8    8    9    8
   23   21   21    8   19    3   24   30   26   22
    7    7    5    4    7    9    8    7    6    7
    1    0    0    0    1    7    2    2    3    1
   13    7    3    4    2    7   11    4    9    9
   18   23   36   18   28   28   37   47   39   50
   24   35   31   51   10   55   66   77   67   61
[torch.ByteTensor of size 1x10x10]

In [119]:
inputs

Variable containing:
(0 ,0 ,.,.) = 
  244  247  238  233  232  218  229  228  214  231
  229  239  245  212  216  228  212  216  210  222
  226  225  233  219  236  222  233  235  234  229
  238  233  242  232  235  227  233  229  230  232
  240  237  226  229  226  234  208  234  218  231
  231  226  232  231  214  219  228  238  223  217
  191  212  191  226  214  159  186  220  211  213
  236  232  227  226  217  185  223  208  216  208
  229  224  228  233  227  219  226  208  217  208
  221  230  217  221  232  225  232  220  202  215
[torch.FloatTensor of size 1x1x10x10]

In [149]:
Variable(torch.arange(0, 2))

Variable containing:
 0
 1
[torch.FloatTensor of size 2]